Installing Required Libraries


In [1]:
!pip install gradio reportlab pillow transformers
!apt-get update && apt-get install -y imagemagick


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.1 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,270 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd

Importing Necessary Libraries

In [2]:
import gradio as gr
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.utils import ImageReader
from PIL import Image
import io
import os
import datetime
from transformers import pipeline


Helper - Resume PDF Generator

In [3]:
def generate_pdf(data, photo=None):
    buffer = io.BytesIO()
    c = canvas.Canvas(buffer, pagesize=letter)
    width, height = letter
    y = height - 50

    c.setFont("Helvetica-Bold", 16)
    c.drawString(50, y, data['name'])
    y -= 20
    c.setFont("Helvetica", 10)
    c.drawString(50, y, data['contact'])
    y -= 30

    if photo:
        try:
            img = Image.open(photo.name)
            img = img.resize((80, 100))
            img_io = io.BytesIO()
            img.save(img_io, format='PNG')
            img_io.seek(0)
            c.drawImage(ImageReader(img_io), 450, y + 10, width=80, height=100)
        except Exception as e:
            print("Photo error:", e)

    def draw_section(title, content_list):
        nonlocal y
        c.setFont("Helvetica-Bold", 12)
        c.drawString(50, y, title)
        y -= 15
        c.setFont("Helvetica", 10)
        for line in content_list:
            c.drawString(60, y, f"- {line}")
            y -= 12
        y -= 10

    draw_section("Summary", [data['summary']])
    draw_section("Education", data['education'])
    draw_section("Experience", data['experience'])
    draw_section("Skills", data['skills'].split(','))
    if data['projects']:
        draw_section("Projects", data['projects'])

    c.save()
    buffer.seek(0)
    return buffer


Gradio UI - Resume Form

In [4]:
import gradio as gr

def create_resume(name, contact, summary, education, experience, skills, projects, photo):
    data = {
        "name": name,
        "contact": contact,
        "summary": summary,
        "education": education.strip().split('\n'),
        "experience": experience.strip().split('\n'),
        "skills": skills,
        "projects": projects.strip().split('\n') if projects else [],
    }
    pdf_buffer = generate_pdf(data, photo)
    output_path = "resume_output.pdf"
    with open(output_path, "wb") as f:
        f.write(pdf_buffer.getvalue())
    return output_path

with gr.Blocks(title="Resume Builder UI") as demo:
    gr.Markdown("## Professional Resume Builder\nCreate a professional PDF resume with your information.")

    with gr.Row():
        with gr.Column():
            name = gr.Textbox(label="Full Name", placeholder="Enter your name")
            contact = gr.Textbox(label="Contact (Email, Phone, LinkedIn)")
            summary = gr.Textbox(label="Summary", lines=3, placeholder="e.g. Experienced AI/ML engineer...")
            education = gr.Textbox(label="Education (one per line)", lines=4)
            experience = gr.Textbox(label="Experience (one per line)", lines=4)
            skills = gr.Textbox(label="Skills (comma-separated)", placeholder="Python, TensorFlow, NLP")
            projects = gr.Textbox(label="Projects (optional, one per line)", lines=3)
            photo = gr.File(label="Upload Profile Photo (optional)", file_types=[".jpg", ".png", ".jpeg"])
            submit_btn = gr.Button("Generate Resume")

        with gr.Column():
            output_file = gr.File(label="Download Your Resume (PDF)")

    submit_btn.click(fn=create_resume,
                     inputs=[name, contact, summary, education, experience, skills, projects, photo],
                     outputs=[output_file])


Launching The Interface

In [ ]:
demo.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://4b90f70281fe874263.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
